In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import openfilegui
import matplotlib.pyplot as plt


In [ ]:
K_PROJECT = 'Project'
K_ACCOUNT = 'Account'
K_PAYMENT_ACCOUNT = 'Payment account'
K_MERCHANT = 'Merchant'
K_ADDRESS = 'Address'
K_NOTE = 'Note'
K_TAGS = 'Tags'
K_AUTHOR = 'Author'
K_IMAGE1 = 'Image 1'
K_IMAGE2 = 'Image 2'
K_IMAGE3 = 'Image 3'
K_DATETIME = 'Date time'
K_TYPE = 'Type'
K_CATEGORY = 'Category'
K_EXPENSE = 'Expense'
K_INCOME = 'Income'
K_TRANSFER = 'Transfer'
K_INVESTING = 'Investing'
K_MONTH = 'Month'
K_AMOUNT = 'Amount'
K_TOTAL = 'Total'
K_CURRENCY_RATE = 'Currency rate (Relative standard currency)'
K_CURRENCY = 'Currency'
K_INDEX = 'index'
K_EXPENSE_SHARE = 'Expense share'
K_ACCOUNT_RECEIVABLE = "Account receivable"
K_INVESTING_SHARE = 'Investing share'
K_OTHER = 'Other'
K_TEMP = 'Temp'
K_PARENT_CATEGORY = 'Parent Category'


In [ ]:
def parseDatetime(x: str):
    rus = {"янв": "jan",
           "февр": "feb", "фев": "feb",
           "мар": "mar",
           "апр": "apr",
           "май": "may", "мая": "may",
           "июн": "jun",
           "июл": "jul",
           "авг": "aug",
           "сент": "sep", "сен": "sep",
           "окт": "oct",
           "нояб": "nov", "ноя": "nov",
           "дек": "dec",
           "г. ": "", 
           ".": ""}
    for r, e in rus.items():
        x = x.lower().replace(r, e)
    try:
        dt = datetime.strptime(x.lower(), u'%d %b %Y %H:%M:%S')
    except:
        dt = datetime.strptime(x.lower(), u'%d%m%Y %H:%M:%S')
    return dt


In [ ]:
file = openfilegui.gui_fname("./data/").decode()


In [ ]:
df = pd.read_csv(file, on_bad_lines="warn", sep="\t")
df = df.drop(columns=[K_PROJECT, K_ACCOUNT, K_PAYMENT_ACCOUNT, K_MERCHANT,
                      K_ADDRESS, K_NOTE, K_TAGS, K_AUTHOR,
                      K_IMAGE1, K_IMAGE2, K_IMAGE3, K_CURRENCY])
df[K_DATETIME] = df[K_DATETIME].apply(lambda x: parseDatetime(x))
df[K_DATETIME] = pd.to_datetime(df[K_DATETIME]).astype(np.int64)
df[K_DATETIME] = df[K_DATETIME].apply(lambda x: x/1000000000)
df[K_AMOUNT] = df[K_AMOUNT].apply(
    lambda x: x.replace(u"\u00A0", '').replace(',', '.'))
df[K_AMOUNT] = df[K_AMOUNT].astype(float)
df[K_CURRENCY_RATE] = df[K_CURRENCY_RATE].apply(lambda x: x.replace(',', '.'))
df[K_CURRENCY_RATE] = df[K_CURRENCY_RATE].astype(float)
df[K_AMOUNT] = df[K_AMOUNT]*df[K_CURRENCY_RATE]
df = df.drop(columns=[K_CURRENCY_RATE])


In [ ]:
start_date = datetime.fromtimestamp(df[K_DATETIME].min())
end_date = datetime.fromtimestamp(df[K_DATETIME].max())


In [ ]:
expense_ = df[df[K_TYPE] == K_EXPENSE]
income_ = df[df[K_TYPE] == K_INCOME]
invest_ = df[(df[K_TYPE] == K_TRANSFER) & (df[K_CATEGORY] == K_INVESTING)]



In [ ]:
def transformByCat(data:pd.DataFrame, level:str, col_name:str):
    if level == "M":
        date_mode = "%m-%Y"
    elif level == "Y":
        date_mode = "%Y"
    elif level == "Q":
        raise NotImplementedError() #TODO quarter
    else:
        raise Exception()

    unique_cats = data[col_name].unique()
    data_by_cat = pd.DataFrame(columns=unique_cats)

    el_list = pd.period_range(start=start_date, end=end_date, freq=level)
    el_list = [el.strftime(date_mode) for el in el_list]
    data_by_cat[K_INDEX] = el_list
    data_by_cat = data_by_cat.set_index(K_INDEX)

    for col in data_by_cat.columns:
        data_by_cat[col].values[:] = 0

    for index, row in data.iterrows():
        date = datetime.fromtimestamp(row[K_DATETIME])
        el = date.strftime(date_mode)
        cat = row[col_name]
        value = row[K_AMOUNT]
        data_by_cat.loc[data_by_cat.index == el, cat] += abs(value)

    data_by_cat[K_TOTAL] = data_by_cat.sum(axis=1)

    return data_by_cat

def merge(expense:pd.DataFrame, income:pd.DataFrame, invest:pd.DataFrame):
    result = pd.DataFrame(index=expense.index)
    result[K_EXPENSE] = expense[K_TOTAL]
    result[K_INCOME] = income[K_TOTAL]
    result[K_INVESTING] = invest[K_TOTAL]
    result[K_TOTAL] = result[K_INCOME] - result[K_EXPENSE] - result[K_INVESTING]
    result[K_TOTAL] = result[K_TOTAL].cumsum()

    result[K_EXPENSE_SHARE] = result[K_EXPENSE] / result[K_INCOME] * 100
    result[K_INVESTING_SHARE] = result[K_INVESTING] / result[K_INCOME] * 100
    return result


def getDFForPie(data: pd.DataFrame, year: str, limit=5):
    df = data.T.drop(index=K_TOTAL)
    df = df[[year]]
    df = df.loc[~(df == 0).all(axis=1)]
    sum = df[year].sum()
    if sum == 0:
        return None
    df[year] = df[year]/sum*100
    df.loc[K_OTHER] = 0
    df[K_TEMP] = False
    for index, row in df.iterrows():
        if row[year] < limit:
            df.loc[K_OTHER, year] += row[year]
            df.loc[index, K_TEMP] = True
    df.loc[K_OTHER, K_TEMP] = False
    df = df.drop(df[df[K_TEMP]].index)
    df = df.drop(columns=K_TEMP)

    return df


def makePies(data: pd.DataFrame):
    years = len(data.T.columns)
    # fig, axes = plt.subplots(nrows=len(data.T.columns), figsize=(30,30))
    for year_num in range(years):
        year = data.T.columns[year_num]
        df = getDFForPie(data, year, 2)
        if df is None:
            continue
        # df.plot(ax=axes[year_num], kind='pie', autopct='%i%%', y=year, legend=False, subplots=True)
        plt.figure(figsize=(40, 40))
        df.plot(kind='pie', autopct='%i%%',
                y=year, legend=False, subplots=True)


In [ ]:
expenseM = transformByCat(expense_, 'M', K_CATEGORY)
incomeM = transformByCat(income_, 'M', K_CATEGORY)
incomeM_PC = transformByCat(income_, 'M', K_PARENT_CATEGORY)
investM = transformByCat(invest_, 'M', K_ACCOUNT_RECEIVABLE)
totalM = merge(expenseM, incomeM, investM)

expenseY = transformByCat(expense_, 'Y', K_CATEGORY)
incomeY = transformByCat(income_, 'Y', K_CATEGORY)
incomeY_PC = transformByCat(income_, 'Y', K_PARENT_CATEGORY)
investY = transformByCat(invest_, 'Y', K_ACCOUNT_RECEIVABLE)
totalY = merge(expenseY, incomeY, investY)


In [ ]:
incomeM_PC.loc[:, incomeM_PC.columns != K_TOTAL].plot()
plt.grid(True)

In [ ]:
totalM.loc[:, totalM.columns != K_TOTAL].plot()
plt.grid(True)

In [ ]:
ax = totalY.iloc[:, totalY.columns != K_TOTAL].plot(kind='bar')
totalY[K_EXPENSE_SHARE].plot(ax=ax, secondary_y=True, color='r', ylim=(0,100))
totalY[K_INVESTING_SHARE].plot(ax=ax, secondary_y=True, color='y', ylim=(0,100))
plt.grid(True)

In [ ]:
makePies(expenseY)

In [ ]:
makePies(incomeY)

In [ ]:
makePies(investY)

In [ ]:
incomeY_PC.iloc[:, incomeY_PC.columns != K_TOTAL].plot(kind='barh', stacked=True)
plt.grid(True)

In [ ]:
totalM.loc[:, totalM.columns == K_TOTAL].plot()
plt.grid(True)

In [ ]:
total_income = totalY[K_INCOME].sum()
total_expense = totalY[K_EXPENSE].sum()
total_invest = totalY[K_INVESTING].sum()

print("Total: {:,}".format(total_income - total_expense))
print("Free: {:,}".format(total_income - total_expense - total_invest))
print("Invested: {:,}".format(total_invest))
